## Solving set of polynomial equations using Homotopy Continuation

In the following example I have shown how this can be done,  ideally we would want the number of equations to be the same as the number of variables. When this is true n = length of list of input functions. To be on the safer side though, we can enter n manually or depending the the number of point masses we solve for (then n becomes 2n).

In [5]:
import Pkg; Pkg.add("DynamicPolynomials")
import Pkg; Pkg.add("HomotopyContinuation")

using HomotopyContinuation
using LinearAlgebra
using DynamicPolynomials

   Resolving package versions...
    Updating `C:\Users\Maan\.julia\environments\v1.10\Project.toml`
  [7c1d4256] + DynamicPolynomials v0.6.1
  No Changes to `C:\Users\Maan\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Maan\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Maan\.julia\environments\v1.10\Manifest.toml`


In [6]:

n = 4 # number of unknown variables

# Define parameters
δ = 0.1    # damping coefficient
ω = 1.5    # frequency
α = 1.0    # linear stiffness
β = 0.04   # nonlinearity
γ = 1.0    # driving force amplitude

# declare the variables 
for i in 1:n÷2
    @polyvar u[i] v[i]
end
@polyvar f[1:n]

# this input funcs will take Hew's function as an input containing all the polynomial equations
input_funcs = [(ω^2 - α)*u1 - δ*ω*v1 + γ - (3/4)*β*(u1^3 + u1*v1^2), ω*δ*u1 + (ω^2 - α)*v1 - (3/4)*β*(v1^3 + u1^2*v1)]

# define the variable f1 to have eqn 1 and f2 to have eqn 2
f = input_funcs

# Define the system as a polynomial system
system = HomotopyContinuation.System(f)

# solve the system
result = HomotopyContinuation.solve(system)

# # display non singular solutions
# for sol in result
#     if HomotopyContinuation.is_nonsingular(sol)
#         println(sol)
#     end
# end

# display real solutions
for sol in result
    if HomotopyContinuation.is_real(sol)
        println(sol)
    end
end



LoadError: UndefVarError: `u1` not defined

This was run with 1 mass, hence one set of parameter values, in the case this is not true we can write a function to take a list containing these values and then later use in the polynomial equations. 

In [7]:
n = 4 # number of unknown variables

# declare the variables 
@polyvar u[1:n÷2] v[1:n÷2]
@polyvar f[1:n]

# declare the parameters
@polyvar α[1:n÷2]
@polyvar β[1:n÷2]
@polyvar γ[1:n÷2]
@polyvar δ[1:n÷2]


# this input funcs will take Hew's function as an input containing all the polynomial equations
input_alpha = [1.0, 1.1]
input_beta = [0.04, 0.03]
input_gamma = [1.0, 1.1]
input_delta = [0.1, 0.2]
input_omega = [1.5]

α = input_alpha
β = input_beta
γ = input_gamma
δ = input_delta
ω = input_omega

input_function = [
    (ω[1]^2 - α[1])*u[1] - δ[1]*ω[1]*v[1] + γ[1] - (3/4)*β[1]*(u[1]^3 + u[1]*v[1]^2),
    (ω[1]^2 - α[2])*u[2] - δ[2]*ω[1]*v[2] + γ[2] - (3/4)*β[2]*(u[2]^3 + u[2]*v[2]^2),
    ω[1]*δ[1]*u[1] + (ω[1]^2 - α[1])*v[1] - (3/4)*β[1]*(v[1]^3 + u[1]^2*v[1]),
    ω[1]*δ[2]*u[2] + (ω[1]^2 - α[2])*v[2] - (3/4)*β[2]*(v[2]^3 + u[2]^2*v[2])
]

f = input_function

# Define the system as a polynomial system
system = HomotopyContinuation.System(f)

# solve the system
result = HomotopyContinuation.solve(system)

# display real solutions
for sol in result
    if HomotopyContinuation.is_real(sol)
        println(sol)
    end
end









Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:06
  # paths tracked:                  81
  # non-singular solutions (real):  9 (3)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         9 (3)
PathResult:
 • return_code → :success
 • solution → ComplexF64[-0.8007933510765409 + 0.0im, -0.909297209414843 - 1.793662034335766e-43im, 0.09761994677037714 - 2.802596928649634e-45im, 0.2413880779738752 + 0.0im]
 • accuracy → 2.0201e-16
 • residual → 4.1633e-17
 • condition_jacobian → 4.9136
 • steps → 76 / 0
 • extended_precision → false
 • path_number → 55

PathResult:
 • return_code → :success
 • solution → ComplexF64[-2.0000000000000075 - 2.7733391199176196e-32im, -0.9092972094148429 + 0.0im, 5.999999999999997 - 6.548161810916602e-33im, 0.2413880779738752 + 0.0im]
 • accuracy → 3.7063e-15
 • residual → 1.7764e-15
 • condition_jacobian → 85.588
 • steps → 85 / 1
 • extended_precision → false
 • path_number → 58

PathResult:
 • return_code → :success

### Function Implementation ###

Check PolynomEqSolverFn.jl if you wish to import the function.

In [1]:
function solve_polynomial_system(input_alpha, input_beta, input_gamma, input_delta, input_omega, input_funcs, returnnonsingular=false)
    n = length(input_alpha) * 2  # number of unknown variables

    # Declare variables
    @polyvar u[1:n÷2] v[1:n÷2]

    # Assign parameters
    α = input_alpha
    β = input_beta
    γ = input_gamma
    δ = input_delta
    ω = input_omega

    # Define the system of equations
    f = input_funcs

    # Define the system as a polynomial system
    system = HomotopyContinuation.System(f)

    # Solve the system
    result = HomotopyContinuation.solve(system)

    # Collect real solutions
    real_solutions = []
    for sol in result
        if HomotopyContinuation.is_real(sol)
            push!(real_solutions, sol)
        end
    end

    # Return nonsingular solutions if requested
    if returnnonsingular
        nonsingular_solutions = []
        for sol in real_solutions
            if HomotopyContinuation.is_nonsingular(sol)
                push!(nonsingular_solutions, sol)
            end
        end
        return real_solutions, nonsingular_solutions
    end

    return real_solutions
end

LoadError: LoadError: UndefVarError: `@polyvar` not defined
in expression starting at In[1]:5